In [93]:
import string
import re
import numpy as np
from numpy.linalg import norm
import pandas as pd
from collections import Counter, OrderedDict

import seaborn as sns
import matplotlib.pyplot as plt

!pip install nltk
!pip install contractions
import contractions
import nltk

import sys  
from nltk.tokenize import word_tokenize
from random import shuffle, seed
seed(0)
from sklearn.feature_extraction import DictVectorizer
from nltk.stem import PorterStemmer

from nltk.stem.wordnet import WordNetLemmatizer
!{sys.executable} -m pip install contractions
import contractions
from nltk import word_tokenize, pos_tag

from sklearn.feature_extraction.text import TfidfVectorizer

nltk.download('wordnet')
nltk.download('omw-1.4')

import nltk
from nltk.corpus import stopwords 
sw_nltk = nltk.corpus.stopwords.words('english')


from nltk.corpus import wordnet as wn

from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet
from collections import defaultdict

from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer

from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2

from nltk.sentiment.vader import SentimentIntensityAnalyzer
nltk.download('vader_lexicon')

from nltk.util import bigrams
%matplotlib inline
pd.options.display.max_colwidth=500

[nltk_data] Downloading package wordnet to /home/jovyan/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /home/jovyan/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /home/jovyan/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


In [94]:
# Load in training data and display in pandas dataframe
train_path='training.csv'
all_train_data = pd.read_csv(train_path,  delimiter="\t", skip_blank_lines = True)
test_path ='test.csv'
test_data = pd.read_csv(test_path,  delimiter="\t", skip_blank_lines = True)



In [95]:
epsiode_scene_column = all_train_data.Episode.astype(str) + "-" + all_train_data.Scene.astype(str)
all_train_data['episode_scene'] = epsiode_scene_column
episode_scenes = sorted(list(set([x for x in epsiode_scene_column.values]))) # set function is random, need to sort!

shuffle(episode_scenes)

print(len(episode_scenes))
episode_split = int(0.9*len(episode_scenes))
training_ep_scenes = episode_scenes[:episode_split]
test_ep_scenes = episode_scenes[episode_split:]
print(len(training_ep_scenes), len(test_ep_scenes))

def train_or_heldout_eps(val):
    if val in training_ep_scenes:
        return "training"
    return "heldout"

all_train_data['train_heldout'] = all_train_data['episode_scene'].apply(train_or_heldout_eps)

1394
1254 140


In [96]:
print('Raw Data: ',np.shape(all_train_data))
train_data = all_train_data[all_train_data['train_heldout']=='training']
val_data = all_train_data[all_train_data['train_heldout']=='heldout']
print('Train set: ',np.shape(train_data))
print('Validation set: ',np.shape(val_data))

Raw Data:  (15319, 8)
Train set:  (13638, 8)
Validation set:  (1681, 8)


# Q3. Add dialogue context data and features 

In [97]:
def create_character_document_from_dataframe(df, max_line_count):
    
    character_docs = {}
    character_lc = {}
    sceneinfo = ''
    lines = set()
    for character_line, character_name, character_scene_info, character_episode, character_scene in zip(df.Line, df.Character_name, df.Scene_info, df.Episode, df.Scene):
        if not character_name in character_docs.keys():
            character_docs[character_name] = ""
            character_lc[character_name] = 0
            character_scene_info = character_scene
        if character_lc[character_name]==max_line_count:
            continue
        character_docs[character_name] += str(character_episode) + " --EPISODES-- "
        character_docs[character_name] += str(character_scene) + "--SCENES-- "
        character_docs[character_name] += str(character_scene_info) + "--SCENE_INFOORMATION-- "
        character_docs[character_name] += str(character_line) + " EOL "  # adding an end-of-line token
        character_lc[character_name]+=1
    #for key,value in character_docs.items():    
        #print(key,value)
    print("lines per character", character_lc)
    return character_docs

In [98]:
train_character_docs = create_character_document_from_dataframe(train_data, max_line_count=360)
print('Num. Characters: ',len(train_character_docs.keys()),"\n")
total_words = 0
for name in train_character_docs.keys():
    print(name, 'Number of Words: ',len(train_character_docs[name].split()))
    total_words += len(train_character_docs[name].split())
print("total words", total_words)

lines per character {'SHIRLEY': 360, 'OTHER': 360, 'JACK': 360, 'RONNIE': 360, 'TANYA': 360, 'SEAN': 360, 'ROXY': 360, 'HEATHER': 360, 'MAX': 360, 'IAN': 360, 'JANE': 360, 'STACEY': 360, 'PHIL': 360, 'MINTY': 360, 'CHRISTIAN': 342, 'CLARE': 352}
Num. Characters:  16 

SHIRLEY Number of Words:  6563
OTHER Number of Words:  5964
JACK Number of Words:  7297
RONNIE Number of Words:  6327
TANYA Number of Words:  6921
SEAN Number of Words:  6738
ROXY Number of Words:  6615
HEATHER Number of Words:  6881
MAX Number of Words:  7124
IAN Number of Words:  6947
JANE Number of Words:  6321
STACEY Number of Words:  6637
PHIL Number of Words:  6440
MINTY Number of Words:  6782
CHRISTIAN Number of Words:  6181
CLARE Number of Words:  6982
total words 106720


In [99]:
def reverse_case(match_obj):
    char_elem = match_obj.group(0)
    if char_elem.isupper():
        return char_elem.lower()
    else:
        return char_elem.lower()



In [100]:
def regexStrip(theString, stripChar='\s'):
    theRegex = re.compile(f'^({stripChar})*|({stripChar})*$')
    stripContext = theRegex.sub('', theString)
    return stripContext

In [101]:
def decontracted(phrase):
        phrase = re.sub(r"won\'t", "will not", phrase)
        phrase = re.sub(r"can\'t", "can not", phrase)

        phrase = re.sub(r"n\'t", " not", phrase)
        phrase = re.sub(r"\'re", " are", phrase)
        phrase = re.sub(r"\'s", " is", phrase)
        phrase = re.sub(r"\'d", " would", phrase)
        phrase = re.sub(r"\'ll", " will", phrase)
        phrase = re.sub(r"\'t", " not", phrase)
        phrase = re.sub(r"\'ve", " have", phrase)
        phrase = re.sub(r"\'m", " am", phrase)
        return phrase 

# Q1. Improve pre-processing

In [102]:
def pre_process(character_text):
    
 
    characters = character_text.lower() # lower case
    
    characters= re.sub('[^\w\s]','',characters) #Removing Punctuation


    character_text = regexStrip(characters)  #remove end of line
    sent1 = decontracted(character_text) # decompressing small words in full words
    sent1 = contractions.fix(character_text)  
    
    token = word_tokenize(sent1)
    tokens_without_stopwwords = [word for word in token if not word in sw_nltk]


    
    lemmatizer = WordNetLemmatizer()
    lemiz_sent = []
    tags_map = defaultdict(lambda : wn.NOUN)
    tags_map['N'] = wn.NOUN
    tags_map['A'] = wn.ADJ
    tags_map['V'] = wn.VERB
    tags_map['R'] = wn.ADV
    for token, tag in nltk.pos_tag(tokens_without_stopwwords):
        lemma = lemmatizer.lemmatize(token, tags_map[tag[0]])
        lemiz_sent.append(lemma) 
    return lemiz_sent

In [103]:
training_corpus = [(name, pre_process(doc)) for name, doc in sorted(train_character_docs.items())]


In [104]:
train_labels = [name for name, doc in training_corpus]

# Q2. Improve linguistic feature extraction

In [105]:
from nltk.tokenize import word_tokenize
from nltk.util import ngrams
from nltk.tag import CRFTagger

def get_ngrams(text, n ):
    n_grams = ngrams(text, n)
    return [ ' '.join(grams) for grams in n_grams]

In [106]:
def listToString(s): 
    
    # initialize an empty string
    str1 = " " 
    
    # return string  
    return (str1.join(s))

In [107]:
def to_feature_vector_dictionary(character_doc, extra_features=[]):
    
    counts = Counter(character_doc)  # for now a simple count
    counts = dict(counts)

    n_bigrams = get_ngrams(character_doc, 2 )
    new_bigrams = ['@'.join(n_bigrams) for n_bigrams in n_bigrams]
    counts.update(Counter(new_bigrams)) 
    
   
    pos_tagger = nltk.pos_tag(character_doc)
    pos_tagged = [str(w)+ "@" + str(t) for w, t in pos_tagger]
    
    # make pos_tagged_words a string
    new_pos_tagged_words = [str(pos_tagged) for pos_tagged in pos_tagged]
    pos_counts = Counter(new_pos_tagged_words)
    pos_counts = dict(pos_counts)
    counts.update(pos_counts)

    # sentiment analysis
    sentiment_analyzer = SentimentIntensityAnalyzer()
    character_doc = ' '.join(character_doc)
    sentiment_scores = sentiment_analyzer.polarity_scores(character_doc)
    sentiment_scores = dict(sentiment_scores)
    counts.update(sentiment_scores)
    
    for feature in extra_features:
        counts[feature] += 1

    return counts



# Q4. Improve the vectorization method 

In [108]:
corpusVectorizer = DictVectorizer() 
tfidf_transformer = TfidfTransformer()
def create_document_matrix_from_corpus(corpus, fitting=False):
    
    # uses the global variable of the corpus Vectorizer to improve things
    if fitting:
        corpusVectorizer.fit([to_feature_vector_dictionary(doc) for name, doc in corpus])
    doc_feature_matrix = corpusVectorizer.transform([to_feature_vector_dictionary(doc) for name, doc in corpus])
     # use tf-idf transformer to improve things
    
    if fitting:
        tfidf_transformer.fit(doc_feature_matrix)
    doc_feature_matrix = tfidf_transformer.transform(doc_feature_matrix)
    
    if fitting:
        selector = SelectKBest(score_func=chi2, k='all')
        selector.fit(doc_feature_matrix, train_labels)
        doc_feature_matrix = selector.transform(doc_feature_matrix)
    
    #training_feature_matrix[0].toarray()
    return doc_feature_matrix

training_feature_matrix = create_document_matrix_from_corpus(training_corpus, fitting=True)

In [109]:
# get the validation data- only 40 lines used for each character
val_character_docs = create_character_document_from_dataframe(val_data, max_line_count=40)
print('Num. Characters: ',len(val_character_docs.keys()),"\n")
total_words = 0
for name in val_character_docs.keys():
    print(name, 'Num of Words: ',len(val_character_docs[name].split()))
    total_words += len(val_character_docs[name].split())
print("total words", total_words)

# create list of pairs of (character name, pre-processed character) 
val_corpus = [(name, pre_process(doc)) for name, doc in sorted(val_character_docs.items())]
val_labels = [name for name, doc in val_corpus]

lines per character {'TANYA': 40, 'MAX': 40, 'SEAN': 35, 'SHIRLEY': 40, 'OTHER': 40, 'STACEY': 40, 'RONNIE': 40, 'JACK': 40, 'PHIL': 40, 'IAN': 40, 'JANE': 40, 'ROXY': 40, 'HEATHER': 40, 'MINTY': 40, 'CHRISTIAN': 40, 'CLARE': 40}
Num. Characters:  16 

TANYA Num of Words:  763
MAX Num of Words:  1050
SEAN Num of Words:  636
SHIRLEY Num of Words:  628
OTHER Num of Words:  689
STACEY Num of Words:  702
RONNIE Num of Words:  764
JACK Num of Words:  619
PHIL Num of Words:  785
IAN Num of Words:  789
JANE Num of Words:  738
ROXY Num of Words:  691
HEATHER Num of Words:  693
MINTY Num of Words:  753
CHRISTIAN Num of Words:  791
CLARE Num of Words:  699
total words 11790


In [110]:
def compute_cosine_similarity(v1, v2):
    
    
    # compute cosine similarity manually
    manual_cosine_similarity = np.dot(v1, v2)  /(norm(v1) * norm(v2))
    
    return manual_cosine_similarity

In [111]:
def compute_IR_evaluation_scores(train_feature_matrix, test_feature_matrix, train_labels, test_labels):
    
    rankings = []
    all_cosine_similarities = []
    pairwise_cosine_similarity = []
    pairs = []
    correct = 0
    for i, target in enumerate(test_labels):
        # compare the left out character against the mean
        idx = i 
        fm_1 = test_feature_matrix.toarray()[idx]
        all_sims = {}
        # print("target:", target)
        for j, other in enumerate(train_labels):
            fm_2 = train_feature_matrix.toarray()[j]
            manual_cosine_similarity = compute_cosine_similarity(fm_1, fm_2)
            pairs.append((target, other))
            pairwise_cosine_similarity.append(manual_cosine_similarity)
            if other == target:
                all_cosine_similarities.append(manual_cosine_similarity)
            all_sims[other] = manual_cosine_similarity

            # print(target, other, manual_cosine_similarity)
        sorted_similarities = sorted(all_sims.items(),key=lambda x:x[1],reverse=True)
        # print(sorted_similarities)
        ranking = {key[0]: rank for rank, key in enumerate(sorted_similarities, 1)}
        # print("Ranking for target", ranking[target])
        if ranking[target] == 1:
            correct += 1
        rankings.append(ranking[target])
        # print("*****")
    mean_rank = np.mean(rankings)
    mean_cosine_similarity = np.mean(all_cosine_similarities)
    accuracy = correct/len(test_labels)
    print("mean rank", np.mean(rankings))
    print("mean cosine similarity", mean_cosine_similarity)
    print(correct, "correct out of", len(test_labels), "/ accuracy:", accuracy )
    
    # get a dafaframe showing all the similarity scores of training vs test docs
    df = pd.DataFrame({'doc1': [x[0] for x in pairs], 'doc2': [x[1] for x in pairs],
                       'similarity': pairwise_cosine_similarity})

    # display characters which are most similar and least similar
    df.loc[[df.similarity.values.argmax(), df.similarity.values.argmin()]]
    return (mean_rank, mean_cosine_similarity, accuracy, df)

In [112]:
val_feature_matrix = create_document_matrix_from_corpus(val_corpus, fitting=False)

In [113]:
val_feature_matrix

<16x29182 sparse matrix of type '<class 'numpy.float64'>'
	with 7082 stored elements in Compressed Sparse Row format>

In [114]:
mean_rank, mean_cosine_simliarity, acc, df = compute_IR_evaluation_scores(training_feature_matrix, val_feature_matrix, train_labels, val_labels)

mean rank 1.5625
mean cosine similarity 0.7417053266118963
11 correct out of 16 / accuracy: 0.6875


# Q5. Select and test the best vector representation method

In [115]:
train_character_docs = create_character_document_from_dataframe(all_train_data, max_line_count=400)
print('Num. Characters: ',len(train_character_docs.keys()),"\n")
total_words = 0
for name in train_character_docs.keys():
    print(name, 'Number of Words: ',len(train_character_docs[name].split()))
    total_words += len(train_character_docs[name].split())
print("total words", total_words)

training_corpus = [(name, pre_process(doc)) for name, doc in train_character_docs.items()]
train_labels = [name for name, doc in training_corpus]

corpusVectorizer = DictVectorizer()   # initialize a corpusVectorizor which will output sparse vectors from dicts


training_feature_matrix = create_document_matrix_from_corpus(training_corpus, fitting=True)

# get the test data using 40 lines per character
test_character_docs = create_character_document_from_dataframe(test_data, max_line_count=40)
print('Num. Characters: ',len(test_character_docs.keys()),"\n")
total_words = 0
for name in test_character_docs.keys():
    print(name, 'Number of Words: ',len(test_character_docs[name].split()))
    total_words += len(test_character_docs[name].split())
print("total words", total_words)

# create list of pairs of (character name, pre-processed character) 
test_corpus = [(name, pre_process(doc)) for name, doc in test_character_docs.items()]
test_labels = [name for name, doc in test_corpus]


# Just transform the val_feature_matrix, don't fit
test_feature_matrix = create_document_matrix_from_corpus(test_corpus, fitting=False)


mean_rank, mean_cosine_simliarity, acc, df = compute_IR_evaluation_scores(training_feature_matrix, test_feature_matrix, train_labels, test_labels)





lines per character {'SHIRLEY': 400, 'OTHER': 400, 'JACK': 400, 'RONNIE': 400, 'TANYA': 400, 'SEAN': 400, 'ROXY': 400, 'HEATHER': 400, 'MAX': 400, 'IAN': 400, 'JANE': 400, 'STACEY': 400, 'PHIL': 400, 'MINTY': 400, 'CHRISTIAN': 385, 'CLARE': 400}
Num. Characters:  16 

SHIRLEY Number of Words:  7255
OTHER Number of Words:  6664
JACK Number of Words:  7920
RONNIE Number of Words:  7065
TANYA Number of Words:  7692
SEAN Number of Words:  7446
ROXY Number of Words:  7279
HEATHER Number of Words:  7563
MAX Number of Words:  8192
IAN Number of Words:  7794
JANE Number of Words:  7082
STACEY Number of Words:  7357
PHIL Number of Words:  7222
MINTY Number of Words:  7449
CHRISTIAN Number of Words:  7019
CLARE Number of Words:  7832
total words 118831
lines per character {'SHIRLEY': 40, 'OTHER': 40, 'HEATHER': 40, 'PHIL': 40, 'SEAN': 40, 'TANYA': 40, 'MAX': 40, 'JACK': 40, 'IAN': 40, 'JANE': 40, 'STACEY': 40, 'ROXY': 40, 'RONNIE': 40, 'CHRISTIAN': 40, 'MINTY': 40, 'CLARE': 40}
Num. Characters: 